In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name: str):
    """
    Web検索に対応したチャットボット用のグラフを作る。

    Args:
        model_name (str): 使用するOpenAIモデル名

    Returns:
        StateGraph: 実行可能なグラフ
    """

    # 検索ツールの定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    # グラフの作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=model_name)
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノード
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # ノード追加
    graph_builder.add_node("chatbot", chatbot)

    # ツールノード追加
    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)

    # ツール呼び出しの有無で遷移先を決定
    graph_builder.add_conditional_edges("chatbot", tools_condition)

    # ツール実行後は再びchatbotへ戻る
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノード設定
    graph_builder.set_entry_point("chatbot")

    # 記憶付きグラフとしてコンパイル
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)

    return graph


# ===== グラフ実行関数 =====
def stream_graph_updates(graph, user_input: str):
    """
    グラフを実行して、最後の回答だけ表示する。
    """
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values",
    )

    last_event = None
    for event in events:
        last_event = event

    if last_event:
        print(last_event["messages"][-1].content, flush=True)


# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# モデル名
MODEL_NAME = "gpt-4o-mini"

# グラフの作成
graph = build_graph(MODEL_NAME)

# メインループ
try:
    while True:
        print()
        message = input("メッセージを入力:")
        if message.strip() == "":
            break

        print(f"{message}")

        # グラフ実行
        stream_graph_updates(graph, message)

except KeyboardInterrupt:
    print("処理を中断しました。")

print("---ご利用ありがとうございました！---")